In [25]:
import requests
import os
from bs4 import BeautifulSoup
import re

from selenium import webdriver
from base64 import b64encode
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import urllib.parse

In [26]:
tagPageUrl = "http://www.collectorcarads.com/advancedsearch.asp"

In [27]:
bigFile = open('fossilcars.txt', 'a', encoding='utf-8')

In [28]:
# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=tagPageUrl)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

# Get the year links
tds = tagPageSoup.find_all('td', class_='makelist')
yearLinkSoup = tds[2]
yearLinks = yearLinkSoup.find_all("a", class_="sidelinks")

In [29]:
# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=tagPageUrl)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

# Get the year column
tds = tagPageSoup.find_all('td', class_='makelist')
yearLinkSoup = tds[2]
yearLinks = yearLinkSoup.find_all("a", class_="sidelinks")

for yearLink in yearLinks:
    url = urllib.parse.urljoin(tagPageUrl, re.sub(' ', '%20', yearLink['href']))
    carPage = requests.get(url=url)
    carTdSoup = BeautifulSoup(carPage.content, 'lxml')
    carTds = carTdSoup.find_all("td", class_="lines")
    if len(carTds):
        carLink = carTds[1].find_all("a")
        if len(carLink):
            carLinkUrl = urllib.parse.urljoin(tagPageUrl, re.sub(' ', '%20', carLink[0]['href']))
            page = requests.get(url=carLinkUrl)
            soup = BeautifulSoup(page.content, 'lxml')

            model = soup.find_all("h1")
            if len(model):
                model = model[0].get_text().strip()
            else:
                model = "Unknown"

            price = soup.find_all("span", class_="pricebold")
            if len(price):
                price = price[0].get_text().strip()
            else:
                price = "Unknown"

            try:
                phone = soup.select("span.adtext > strong")
                if len(phone):
                    phone = phone[0].get_text().strip()
                if len(phone):
                    bigFile.write(model + '\t' + price + '\t' + phone + '\n')
            except:
                pass

In [30]:
bigFile.close()